# Script de procesamiento de datos

a partir de los datos obtenidos desde Whisper via audios de youtube, limpiaremos los datos y los preparemos para la base de datos sqlite.

In [1]:
import pandas as pd
import numpy as np

#data = pd.read_csv("data.csv")

In [14]:
links_2 = ['bGBuz7UNN5I','r1zIkR7IHoc', 'TwzPgh0uqAY','R-B6eGjbl8g','mVUUDWaKteI','feSDV9vTVX8','7q1C7T07Hps','J8d-sdrgcpY','EEXMArUthAc','F3taGIhnFjs',
           'G6HA8GYIe44','chhNIbCrmnI','yIM3zckF2G8','NPGv7DOBfdM','T1eyt_TLr7k','fuXknV1J_pU','32UJCMAHQno','0SQv0CoiO4M','CsOm_-oM4mg','YSTW3T-j3xg',
           'h28WaqvGWk8', '3hd4j_uXhYU','YsomPi0gzsk','PcTrdN7qml4','qwhuAj-epNw','Dv0BYQyvlXc','iEtE967MuGg','03M0H7xayoA','IWDQTC-9qcU','7NYxsDCxQ_4']
s1 = pd.read_csv("data_2_stage_1.csv")
s2 = pd.read_csv("data_2_stage_2.csv") # bGBuz7UNN5I  en s2
s3 = pd.read_csv("data_2_stage_3.csv") # h28WaqvGWk8  en s3

In [27]:
S = pd.concat([s1,s2,s3], ignore_index=True)

In [28]:
pd.set_option('display.max_colwidth', None)

In [29]:
data = S.copy()

In [30]:
def limpiar_puntos(texto):
    if pd.isna(texto): return texto
    partes = str(texto).split('.')
    if len(partes) > 2:
        # Une la primera parte con la segunda mediante un punto, ignora el resto
        return f"{partes[0]}.{partes[1]}"
    return texto

data['start'] = data['start'].apply(limpiar_puntos).astype(float)
data['end'] = data['end'].apply(limpiar_puntos).astype(float)

In [31]:
# limpieza de textos vacíos
data['text'] = data['text'].str.strip()
data = data.replace('', np.nan).dropna()
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8122 entries, 0 to 8460
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  8122 non-null   int64  
 1   id_video    8122 non-null   object 
 2   start       8122 non-null   float64
 3   end         8122 non-null   float64
 4   text        8122 non-null   object 
dtypes: float64(2), int64(1), object(2)
memory usage: 380.7+ KB


In [32]:

# Limpiamos el formato extraño de puntos antes de empezar
data['start'] = pd.to_numeric(data['start'].astype(str).str.replace(r'\.(?=\d{2}\.)', '', regex=True), errors='coerce')
data['end']   = pd.to_numeric(data['end'].astype(str).str.replace(r'\.(?=\d{2}\.)', '', regex=True), errors='coerce')


In [33]:
# intervalo de merge de 2 segundos: start < 2s

def segmentacion_estricta(df):
    bloques = []
    texto_acumulado = []
    tiempo_inicio = None
    
    signos_cierre = ('.', '!', '?', '…')

    for i, row in df.iterrows():
        # Guardamos el inicio del primer segmento del bloque
        if tiempo_inicio is None:
            tiempo_inicio = row['start']
        
        texto_actual = str(row['text']).strip()
        texto_acumulado.append(texto_actual)
        
        # SI el texto termina en puntuación, cerramos el bloque
        if texto_actual.endswith(signos_cierre):
            bloques.append({
                'id_video': row['id_video'],
                'start': tiempo_inicio,
                'end': row['end'],
                'text': " ".join(texto_acumulado)
            })
            # Reiniciamos para el siguiente bloque
            texto_acumulado = []
            tiempo_inicio = None
            
    # Manejo de texto huérfano (si el video termina sin punto final)
    if texto_acumulado:
        bloques.append({
            'id_video': df.iloc[-1]['id_video'],
            'start': tiempo_inicio,
            'end': df.iloc[-1]['end'],
            'text': " ".join(texto_acumulado)
        })
        
    return pd.DataFrame(bloques)



In [34]:
# Uso
df_final = segmentacion_estricta(data)
def generar_url(row):
    # Convertimos a entero porque YouTube no procesa milisegundos en la URL
    segundos = int(row['start'])
    return f"https://www.youtube.com/watch?v={row['id_video']}&t={segundos}s"

# Creamos la nueva columna
df_final['link'] = df_final.apply(generar_url, axis=1)


In [35]:
df_final

,id_video,start,end,text,link
0,bGBuz7UNN5I,0.00,1.68,¿Por qué no hablas por tal cosa? ¿Listo?,https://www.youtube.com/watch?v=bGBuz7UNN5I&t=0s
1,bGBuz7UNN5I,1.90,4.16,¿Cuál es tu estilo como entrevistador?,https://www.youtube.com/watch?v=bGBuz7UNN5I&t=1s
2,bGBuz7UNN5I,4.16,12.08,"A mí me gustan los entrevistadores de programas en que venden productos de gimnasia, que siempre tienen una persona y le van preguntando la fondad del producto.",https://www.youtube.com/watch?v=bGBuz7UNN5I&t=4s
3,bGBuz7UNN5I,12.60,13.68,¿Se acaba de cuenta de eso?,https://www.youtube.com/watch?v=bGBuz7UNN5I&t=12s
4,bGBuz7UNN5I,14.50,15.32,"Pero eso no, un programa entre mí.",https://www.youtube.com/watch?v=bGBuz7UNN5I&t=14s
...,...,...,...,...,...
3984,7NYxsDCxQ_4,777.53,779.97,Te pedí.,https://www.youtube.com/watch?v=7NYxsDCxQ_4&t=777s
3985,7NYxsDCxQ_4,785.15,786.51,Te lo he dicho mil veces.,https://www.youtube.com/watch?v=7NYxsDCxQ_4&t=785s
3986,7NYxsDCxQ_4,786.51,786.63,Eres...,https://www.youtube.com/watch?v=7NYxsDCxQ_4&t=786s
3987,7NYxsDCxQ_4,786.63,786.67,Juan Pablo.,https://www.youtube.com/watch?v=7NYxsDCxQ_4&t=786s


In [39]:
tokensearch = r'(?=.*abueloo)'
res = df_final[df_final['text'].str.contains(tokensearch, regex=True, case=False, na=False)]
res

,id_video,start,end,text,link


In [42]:
df_final.to_csv("data_2.csv")

In [43]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3989 entries, 0 to 3988
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   id_video  3989 non-null   object 
 1   start     3989 non-null   float64
 2   end       3989 non-null   float64
 3   text      3989 non-null   object 
 4   link      3989 non-null   object 
dtypes: float64(2), object(3)
memory usage: 155.9+ KB


In [47]:
data1 = pd.read_csv("data.csv")
data1 = data1[['id_video', 'start', 'end', 'text', 'link']]
data1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11574 entries, 0 to 11573
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   id_video  11574 non-null  object 
 1   start     11574 non-null  float64
 2   end       11574 non-null  float64
 3   text      11574 non-null  object 
 4   link      11574 non-null  object 
dtypes: float64(2), object(3)
memory usage: 452.2+ KB


In [48]:
data = pd.concat([df_final, data1], ignore_index=True)
data.info()
data.to_csv("data_17e.csv")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15563 entries, 0 to 15562
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   id_video  15563 non-null  object 
 1   start     15563 non-null  float64
 2   end       15563 non-null  float64
 3   text      15563 non-null  object 
 4   link      15563 non-null  object 
dtypes: float64(2), object(3)
memory usage: 608.1+ KB


# Risómetro [test]

> es muy pesado ejecutar esto. probar un un subset muy pequeño df_final.iloc[:20]

In [ ]:
#!pip install pydub opencv-python

In [ ]:
def medir_risa_local(id_video, start, end):
    # 1. Construir la ruta al archivo que ya tienes
    ruta_audio = f"audios/video_{id_video}.mp3"
    
    # Verificamos si el archivo existe 
    if not os.path.exists(ruta_audio):
        return None 
    
    # 2. Cargar el audio completo
    audio_completo = AudioSegment.from_mp3(ruta_audio)
    
    # 3. Definir el segmento de la risa (3 segundos después del 'end')
    punto_corte_ms = end * 1000
    fin_risa_ms = (end + 3.36) * 1000
    
    # Extraer el segmento de la reacción
    audio_reaccion = audio_completo[punto_corte_ms : fin_risa_ms]
    
    # 4. Calcular el volumen (dBFS)
    # Si el segmento es puro silencio o muy corto, dBFS devuelve -inf
    volumen = audio_reaccion.dBFS
    
    return volumen

#df_2 = df_final.iloc[:20]
#df_2['puntuacion_risa'] = df_2.apply(lambda r: medir_risa_local(r['id_video'], r['start'], r['end']), axis=1)

In [ ]:
# Ver los 5 chistes más exitosos
mejores_chistes = df_2.sort_values('puntuacion_risa', ascending=False).head(5)

for i, row in mejores_chistes.iterrows():
    print(f"Risa ({row['puntuacion_risa']:.2f} dB): {row['text']}")